In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
import random

# Load the DailyDialog dataset
dataset = load_dataset("daily_dialog")

# Initialize the tokenizer and model for dialogue generation
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

# Initialize a question-answering pipeline with a smaller model for efficiency
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

# Preprocess the dataset to tokenize the dialog
def preprocess_data(examples):
    inputs = []
    attention_masks = []
    for dialog in examples['dialog']:
        dialog_input_ids = []
        dialog_attention_masks = []
        for utterance in dialog:
            tokenized = tokenizer(utterance, padding='max_length', truncation=True, max_length=128)
            dialog_input_ids.append(tokenized['input_ids'])
            dialog_attention_masks.append(tokenized['attention_mask'])
        inputs.append(dialog_input_ids)
        attention_masks.append(dialog_attention_masks)
    return {'input_ids': inputs, 'attention_mask': attention_masks}

tokenized_dataset = dataset.map(preprocess_data, batched=True, remove_columns=["dialog"])

# Fun and engaging responses
fun_responses = {
    "how are you": [
        "I'm doing great! How about you?",
        "I'm here, ready to chat! What's on your mind?",
        "Feeling like a million bytes today! And you?"
    ],
    "tell me a joke": [
        "Why don’t scientists trust atoms? Because they make up everything!",
        "Why did the scarecrow win an award? Because he was outstanding in his field!",
        "I told my computer I needed a break, and now it won’t stop sending me Kit-Kats.",
        "Why was the math book sad? It had too many problems."
    ],
    "hello": [
        "Hi there! How's it going?",
        "Hey! Nice to see you!",
        "Hello! What can I do for you today?"
    ],
    "bye": [
        "Goodbye! Have a great day!",
        "See you later! Take care!",
        "Bye! Don't be a stranger!"
    ],
    "what is your name": [
        "I’m your friendly virtual assistant! What's your name?",
        "You can call me ChatBot! How about you?"
    ],
    "what can you do": [
        "I can chat with you, tell jokes, answer questions, and much more!",
        "I'm here to help you with anything I can! What do you want to talk about?"
    ]
}

# Function to generate responses
def generate_response(input_text, chat_history_ids=None):
    # Convert input text to lowercase for matching
    lower_input = input_text.lower()

    # Check if the input matches any fun response keys
    for key in fun_responses:
        if key in lower_input:
            return random.choice(fun_responses[key]), chat_history_ids

    # Check if the input is a factual question
    if any(lower_input.startswith(q) for q in ["what is", "who is", "where is", "when is", "how", "why"]):
        try:
            result = qa_pipeline(question=input_text, context="Narendra Modi is the Prime Minister of India. Paris is the capital of France. Elon Musk is the CEO of Tesla. The Earth orbits the Sun. Python is a programming language.")
            return result['answer'], chat_history_ids
        except Exception as e:
            return "I'm sorry, I don't have the information to answer that question.", chat_history_ids
    else:
        new_user_input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt')
        if chat_history_ids is not None:
            bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
        else:
            bot_input_ids = new_user_input_ids

        reply_ids = model.generate(
            bot_input_ids,
            max_length=1000,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            no_repeat_ngram_size=2
        )

        chat_history_ids = reply_ids
        response = tokenizer.decode(reply_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

        return response, chat_history_ids

print("Chatbot: Hello! What's your name?")
user_name = input("You: ")
print(f"Chatbot: Nice to meet you, {user_name}! How can I assist you today?")
chat_history_ids = None

while True:
    user_input = input(f"{user_name}: ")
    if user_input.lower() in ["exit", "quit", "bye"]:
        print(f"Chatbot: Goodbye, {user_name}! It was nice chatting with you!")
        break
    response, chat_history_ids = generate_response(user_input, chat_history_ids)
    print(f"Chatbot: {response}")


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Chatbot: Hello! What's your name?
You: hi
Chatbot: Nice to meet you, hi! How can I assist you today?
hi: how are you
Chatbot: I'm just a bunch of code, but I'm doing great! How about you?
hi: its a fun time


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Chatbot: I'm not sure if it's a good or bad thing.
hi: ok
Chatbot: It's not a bad idea. I'm just not going to do it. It's just a little bit of fun. :P
hi: give me the funny joke
Chatbot: You're not funny. You're just sad. And I don't know why. But I do. Because I like you. lt 3
hi: its ajoke
Chatbot: Why is this funny?
hi: no
Chatbot: why is it funny
hi: its not funny
Chatbot: it's funny, but it is funny!
